# Загрузка натюненной модели

Предварительно необходимо загрузить модель в папку. Содержимое папки должно быть приблизительно таким:
```txt
adapter_config.json        README.md                tokenizer.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.model
chat_template.jinja        tokenizer_config.json
```

Необходимо получить токен HF ([ссылка](https://huggingface.co/settings/tokens))

Загрузить переменную окружения с именем `HF_TOKEN` во вкладке Secrets. Вставить в эту перемееную ключ

# Код загрузки модели

In [ ]:
!pip install -U huggingface_hub
!pip install "huggingface-hub>=0.34.0,<1.0"
!pip install bitsandbytes

In [ ]:
import os
import torch
from google.colab import userdata
from huggingface_hub import upload_folder
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import notebook_login, create_repo

### Переменные

Если модель загружана на google disk. Пусть к папке остается таким же.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

USE_GOOGLE_DISK = True

In [ ]:
HF_TOKEN_ENV_VAR = "HF_TOKEN" # Имя переменной окружения

BASE_MODEL_PATH = "путь_где+хранится_модель"
MODEL_PATH = f"./{BASE_MODEL_PATH}" if not USE_GOOGLE_DISK else f"/content/drive/MyDrive/{BASE_MODEL_PATH}"

HF_USERNAME = "имя_на_Hugging_face"
MODEL_NAME = "название_модели"
REPO_ID = f"{HF_USERNAME}/{MODEL_NAME}"

IS_PRIVATE=False

### Загрузка модели

In [ ]:
def upload_model_to_hf():
    token = userdata.get(HF_TOKEN_ENV_VAR)
    if not token:
        raise EnvironmentError(f"Переменная окружения {HF_TOKEN_ENV_VAR} не установлена.")
    create_repo(repo_id=REPO_ID, private=IS_PRIVATE, exist_ok=True)
    upload_folder(
        folder_path=MODEL_PATH,
        repo_id=REPO_ID,
        repo_type="model",
    )
    print(f"Модель успешно загрузилась {REPO_ID}")

upload_model_to_hf()

### Тестирование модели - загрузка отправленной модели из HF

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
import torch
from unsloth import FastLanguageModel

In [ ]:
def load_model_and_tokenizer():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=REPO_ID,
        max_seq_length=512,
        dtype=None,
        load_in_4bit=True,
        force_download=True
    )
    return model, tokenizer

model, tokenizer = load_model_and_tokenizer()

In [ ]:
input_text = "Сгенерируй конфликтную ситуацию"

In [ ]:
def test_uploaded_model():
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    print(f"Входной запрос: {input_text}\n")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Ответ модели: {generated_text[len(input_text):]}\n")

test_uploaded_model()